导入库

In [1]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB


获取数据

In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

数据探索/分析

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


训练集有 891 个示例和 11 个特征 + 目标变量 (survived)。其中 2 个特征是浮点数，5 个是整数，5 个是对象。下面我列出了带有简短说明的功能：
survival:    生存
PassengerId: 乘客的唯一 ID。
pclass:    票务类    
sex:    性别      
Age:    年龄    
sibsp:    兄弟姐妹/配偶登船的     
parch:    家长/儿童登船的     
ticket:    票号    
fare:    票价     
cabin:    客舱号  
embarked:    登船港